In [ ]:
from webScraper import *
import pandas as pd
import csv

<font size="5">
    Step 1: Scrape Latest Articles
</font>

In [ ]:
# Scrape latest articles from https://nos.nl
# Outputs latest articles in 'articles_scrapped.json'
webDataScrape()

<font size="5">
    Step 2: Add to Master Article List
</font>

In [ ]:
# Updates master_article_list.json, using articles_scrapped.json as input
updateMasterList()

In [ ]:
# Check length of master article list
master_article_list = []
with open('master_article_list.json') as json_file:
    master_article_data = json.load(json_file)
    for article in master_article_data:
        master_article_list.append(Article(article['ID'], article['url'], article['title'], article['date_time'], article['text']))
print(len(master_article_list))

<font size="5">
    Step 3: Convert text in Articles to list of words with frequency
</font>

In [ ]:
# Retrieve all articles from master list
master_article_list = []
with open('master_article_list.json') as json_file:
    master_article_data = json.load(json_file)
    for article in master_article_data:
        master_article_list.append(Article(article['ID'], article['url'], article['title'], article['date_time'], article['text']))

In [ ]:
# Retrieve list of words from each article
full_word_list = []
for article in master_article_list:
    full_word_list.extend(article.retrieveWordList())
print(full_word_list)

In [ ]:
# Converts word list to dictionary containing count of each word
word_list_count_dict = {}
for word in full_word_list:
    if word in word_list_count_dict.keys():
        word_list_count_dict[word] = word_list_count_dict[word] + 1
    else:
        word_list_count_dict[word] = 1
print(word_list_count_dict)

In [ ]:
# Convert to Dataframe and sort
word_count_df = pd.DataFrame(columns=['word', 'count'])
for word, count in word_list_count_dict.items():
    word_count_df.loc[len(word_count_df.index)] = [word, count]

word_count_sorted_df = word_count_df.sort_values(by=['count'], ascending=False)

In [ ]:
# Output sorted word list to csv
word_count_sorted_df.to_csv('word_list.csv', index=False)

<font size="5">
    Step 4: Remove Invalid Words and Custom Filtered Words
</font>

In [ ]:
# Remove words from invalid_words.csv - These are either symbol characters, names of people etc.
with open('invalid_words.csv', newline='') as csv_file:
    reader = csv.reader(csv_file)
    invalid_word_list = [element[0] for element in list(reader)]

# Remove already learned words
with open('learned_words.csv', newline='') as csv_file:
    reader = csv.reader(csv_file)
    learned_words_list = [element[0] for element in list(reader)]

# Combine lists
words_to_remove_list = invalid_word_list + learned_words_list

In [ ]:
# Remove words from current dataframe word count list
word_count_filtered_df = word_count_sorted_df.copy()
for remove_word in words_to_remove_list:
    word_count_filtered_df = word_count_filtered_df[word_count_filtered_df.word !=remove_word]


In [ ]:
# Output filtered word list to csv
word_count_filtered_df.to_csv('word_list_filtered.csv', index=False)